In [1]:
import io
import re
import torch
import numpy as np
import pandas as pd

from nltk.tokenize import RegexpTokenizer
from imblearn.over_sampling import RandomOverSampler
from google.colab import files
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from google.colab import drive

In [2]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Tokenizer ViBERT
tokenizer = AutoTokenizer.from_pretrained("FPTAI/vibert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("FPTAI/vibert-base-cased", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/255k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/581M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at FPTAI/vibert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Đọc dữ liệu
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Report1/train_data.csv')
val_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Report1/val_data.csv')

train_texts = train_df['content']
train_labels = train_df['label']
val_texts = val_df['content']
val_labels = val_df['label']

In [5]:
# Tokenize dữ liệu với padding
def tokenize_and_pad(texts, tokenizer, max_length=256):
    encodings = tokenizer(
        texts,
        truncation=True,
        padding='max_length',  # Thêm padding để tất cả các văn bản có cùng chiều dài
        max_length=max_length
    )
    return encodings

train_encodings = tokenize_and_pad(train_texts.tolist(), tokenizer, max_length=256)
val_encodings = tokenize_and_pad(val_texts.tolist(), tokenizer, max_length=256)

In [6]:
# Kiểm tra dữ liệu sau khi token hóa
print(train_encodings.keys())
print(val_encodings.keys())
print(train_encodings['input_ids'][:2])  # Hiển thị một vài mẫu tokenized
print(val_encodings['input_ids'][:2])

train_labels = np.array(train_labels).astype(int)
val_labels = np.array(val_labels).astype(int)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
[[2, 606, 523, 13, 20, 37, 232, 338, 913, 1683, 338, 598, 1303, 889, 830, 1620, 23, 29, 1481, 39, 131, 33, 131, 780, 39, 544, 1208, 1018, 83, 1303, 2094, 788, 37, 232, 338, 1870, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 4056, 1767, 146, 15, 1670, 80, 937

In [7]:
class FakeNewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).squeeze()  # Đảm bảo nhãn có kích thước đúng
        return item

    def __len__(self):
        return len(self.labels)

In [8]:
train_dataset = FakeNewsDataset(train_encodings, train_labels.tolist())
val_dataset = FakeNewsDataset(val_encodings, val_labels.tolist())

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [9]:
# Định nghĩa hàm huấn luyện với các siêu tham số có thể tinh chỉnh
def train_model(learning_rate, weight_decay, num_train_epochs):
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=500,
        weight_decay=weight_decay,
        logging_dir='./logs',
        logging_steps=10,
        eval_strategy="steps",
        eval_steps=50,
        save_steps=50,
        save_total_limit=3,
        load_best_model_at_end=True,
        learning_rate=learning_rate,
        save_strategy="steps",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    trainer.train()

    eval_result = trainer.evaluate()
    return eval_result['eval_loss']

In [ ]:
# Tinh chỉnh siêu tham số với Grid Search
param_grid = {
    'learning_rate': [5e-5, 3e-5, 2e-5],
    'weight_decay': [0.01, 0.015, 0.02],
    'num_train_epochs': [3, 4, 5],
}

best_params = None
best_score = float('inf')

for lr in param_grid['learning_rate']:
    for wd in param_grid['weight_decay']:
        for epochs in param_grid['num_train_epochs']:
            eval_loss = train_model(lr, wd, epochs)
            print(f"Learning rate: {lr}, Weight decay: {wd}, Epochs: {epochs}, Eval loss: {eval_loss}")
            if eval_loss < best_score:
                best_score = eval_loss
                best_params = {'learning_rate': lr, 'weight_decay': wd, 'num_train_epochs': epochs}

print(f"Best params: {best_params}, Best eval loss: {best_score}")

Step,Training Loss,Validation Loss
50,0.692500,0.675243
100,0.611100,0.582131
150,0.454400,0.486903
200,0.392800,0.490496
250,0.343700,0.418399
300,0.319900,0.413821
350,0.440600,0.621622
400,0.295400,0.413692


Learning rate: 5e-05, Weight decay: 0.01, Epochs: 3, Eval loss: 0.41369202733039856


Step,Training Loss,Validation Loss
50,0.077700,0.452331
100,0.147500,0.659866
150,0.004000,0.649273
200,0.054000,0.685936


Learning rate: 5e-05, Weight decay: 0.01, Epochs: 4, Eval loss: 0.45233091711997986


Step,Training Loss,Validation Loss
50,0.015600,0.490776
100,0.153300,0.637261
150,0.004000,0.654129
200,0.059400,0.669242


...

Learning rate: 2e-05, Weight decay: 0.02, Epochs: 3, Eval loss: 1.4224315881729126


Step,Training Loss,Validation Loss
50,0.000000,1.444690
100,0.000000,1.718306
150,0.000000,1.529271
200,0.000000,1.580165


Learning rate: 2e-05, Weight decay: 0.02, Epochs: 4, Eval loss: 1.4446901082992554


Step,Training Loss,Validation Loss
50,0.000000,1.465313
100,0.000000,1.731155
150,0.000000,1.530206
200,0.000000,1.576849


Learning rate: 2e-05, Weight decay: 0.02, Epochs: 5, Eval loss: 1.4653130769729614
Best params: {'learning_rate': 5e-05, 'weight_decay': 0.01, 'num_train_epochs': 3}, Best eval loss: 0.41369202733039856


In [10]:
# Tinh chỉnh siêu tham số với Grid Search
param_grid = {
    'learning_rate': [5e-5, 3e-5, 2e-5],
    'weight_decay': [0.01, 0.015, 0.02],
    'num_train_epochs': [3, 4, 5],
}

best_params = None
best_score = float('inf')

for lr in param_grid['learning_rate']:
    for wd in param_grid['weight_decay']:
        for epochs in param_grid['num_train_epochs']:
            eval_loss = train_model(lr, wd, epochs)
            print(f"Learning rate: {lr}, Weight decay: {wd}, Epochs: {epochs}, Eval loss: {eval_loss}")
            if eval_loss < best_score:
                best_score = eval_loss
                best_params = {'learning_rate': lr, 'weight_decay': wd, 'num_train_epochs': epochs}

print(f"Best params: {best_params}, Best eval loss: {best_score}")

Step,Training Loss,Validation Loss
50,0.685300,0.681263
100,0.616900,0.594982
150,0.474700,0.494967
200,0.354400,0.515311
250,0.351400,0.504582
300,0.438000,0.411987
350,0.399400,0.527910
400,0.755200,0.577976


Learning rate: 5e-05, Weight decay: 0.01, Epochs: 3, Eval loss: 0.41198650002479553


Step,Training Loss,Validation Loss
50,0.149300,0.517835
100,0.110700,0.550152
150,0.081000,0.707606
200,0.154100,0.689808


Learning rate: 5e-05, Weight decay: 0.01, Epochs: 4, Eval loss: 0.5178353190422058


Step,Training Loss,Validation Loss
50,0.125900,0.578604
100,0.131900,0.569962
150,0.107700,0.736255
200,0.119600,0.782005
250,0.007300,0.798170


...

Learning rate: 2e-05, Weight decay: 0.02, Epochs: 3, Eval loss: 1.3062816858291626


Step,Training Loss,Validation Loss
50,0.000000,1.337546
100,0.000000,1.453736
150,0.000000,1.519193
200,0.000100,1.608763


Learning rate: 2e-05, Weight decay: 0.02, Epochs: 4, Eval loss: 1.3375463485717773


Step,Training Loss,Validation Loss
50,0.000000,1.366242
100,0.000000,1.476232
150,0.000000,1.544891
200,0.004300,1.637882


Learning rate: 2e-05, Weight decay: 0.02, Epochs: 5, Eval loss: 1.3662420511245728
Best params: {'learning_rate': 5e-05, 'weight_decay': 0.01, 'num_train_epochs': 3}, Best eval loss: 0.41198650002479553


In [24]:
# Khởi tạo Trainer với Dừng sớm (Early Stopping)
training_args_with_early_stop = TrainingArguments(
    output_dir='./results_with_early_stop',
    num_train_epochs=best_params['num_train_epochs'],
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=best_params['weight_decay'],
    logging_dir='./logs_with_early_stop',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    save_total_limit=3,
    load_best_model_at_end=True,
    learning_rate=best_params['learning_rate'],
)

trainer_with_early_stop = Trainer(
    model=model,
    args=training_args_with_early_stop,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
trainer_with_early_stop.train()

Step,Training Loss,Validation Loss
50,0.000000,1.100752
100,0.000000,1.122289
150,0.000000,0.897479
200,0.000300,0.954963
250,0.000100,0.987013
300,0.000000,1.113304
350,0.230200,1.460116


TrainOutput(global_step=350, training_loss=0.024855872427329555, metrics={'train_runtime': 232.3398, 'train_samples_per_second': 14.513, 'train_steps_per_second': 1.821, 'total_flos': 367303033282560.0, 'train_loss': 0.024855872427329555, 'epoch': 2.482269503546099})

In [26]:
# Dự đoán nhãn cho tập kiểm tra
predictions2 = trainer_with_early_stop.predict(val_dataset)

# Lấy nhãn dự đoán từ logits
pred_labels2 = np.argmax(predictions2.predictions, axis=1)

In [27]:
# In ra một số dự đoán để kiểm tra
print(pred_labels2[:10])  # In ra 10 dự đoán đầu tiên
print(pred_labels2[:])  # In ra dự đoán

[0 0 0 1 0 0 1 0 1 0]
[0 0 0 1 0 0 1 0 1 0 1 1 1 0 1 0 0 0 0 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 1 1 1
 0 1 1 1 0 0 1 0 1 0 1 0 1 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0 1 1 0 1 1 0 1 1
 1 1 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 1 1 1 0 1 0 0 0 0 1 1 0
 0 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 1 0 0 0 0 1 1 1 1 0 1 1 1 0 0 1 0 0 0 1
 1 0 1 1 0 0 0 0 1 1 1 1 0 1 0 1 0 1 0 0 1 1 0 1 1 1 0 0 1 1 1 0 1 1 1 1 0
 0 0 0 0 0 1 0 0 1 1 0 1 1 0 1 0 1 0 1 1 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0
 0 1 1 1 0 1 1 1 1 0 0 1 0 1 1 1 0 1 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 1 0 0
 1 0 1 0 1 0 1 1 1 1 0 1 0 0 1 0 1 1 1 1 0 1 1]


In [28]:
# Tính các chỉ số
accuracy2 = accuracy_score(val_labels, pred_labels2)
precision2 = precision_score(val_labels, pred_labels2, pos_label=0)
recall2 = recall_score(val_labels, pred_labels2, pos_label=0)
f12 = f1_score(val_labels, pred_labels2, pos_label=0)
auc2 = roc_auc_score(val_labels, predictions2.predictions[:, 1])

print(f"Accuracy: {accuracy2:.6f}")
print(f"Precision: {precision2:.6f}")
print(f"Recall: {recall2:.6f}")
print(f"F1 Score: {f12:.6f}")
print(f'AUC: {auc2:.6f}')

Accuracy: 0.893617
Precision: 0.901408
Recall: 0.888889
F1 Score: 0.895105
AUC: 0.950584


In [29]:
# Tính các chỉ số
accuracy2 = accuracy_score(val_labels, pred_labels2)
precision2 = precision_score(val_labels, pred_labels2, pos_label=1)
recall2 = recall_score(val_labels, pred_labels2, pos_label=1)
f12 = f1_score(val_labels, pred_labels2, pos_label=1)
auc2 = roc_auc_score(val_labels, predictions2.predictions[:, 1])

print(f"Accuracy: {accuracy2:.6f}")
print(f"Precision: {precision2:.6f}")
print(f"Recall: {recall2:.6f}")
print(f"F1 Score: {f12:.6f}")
print(f'AUC: {auc2:.6f}')

Accuracy: 0.893617
Precision: 0.885714
Recall: 0.898551
F1 Score: 0.892086
AUC: 0.950584
